<a href="https://colab.research.google.com/github/ipeirotis/datasets/blob/master/Create_a_better_Music_database_in_MySQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
!pip install -U -q pip 
!pip install -U -q statsmodels xlrd PyMySQL sqlalchemy sql_magic prophet fastcluster

!pip install names

In [ ]:
import pandas as pd
import numpy as np

from  tqdm import tqdm

import random


import names
from sqlalchemy import create_engine

In [ ]:
conn_string = 'mysql+pymysql://{user}:{password}@{host}/?charset=utf8mb4'.format(
    host = 'db.ipeirotis.org', 
    user = 'root',
    password = 'ae6jQniBS5muV27', #os.environ['MYSQL_PASSWORD'],
    encoding = 'utf8mb4')

engine = create_engine(conn_string)



In [ ]:
# The database
db_name = 'music'

# And lets switch to the database
sql = f"USE {db_name}"
engine.execute(sql)

In [ ]:
for i in range(10):
  n = names.get_full_name()


In [ ]:


results = []
for user_id in range(10):
  g = random.choice(('male', 'female'))
  entry = {
      'user_id': 1+user_id,
    'gender': g,
    'first_name': names.get_first_name(gender=g),
    'last_name': names.get_last_name()  
  }
  results.append(entry)

df = pd.DataFrame(results)
df

In [ ]:
drop_table = "DROP TABLE IF EXISTS music.users;"
engine.execute(drop_table)

create_table_sql = f'''
CREATE TABLE music.`users` (
  `user_id` smallint,
  `gender` char(6),
  `first_name` varchar(50),
  `last_name` varchar(50),
  PRIMARY KEY(user_id)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4
'''
engine.execute(create_table_sql)

with engine.connect() as con:
  df.to_sql(
          name = 'users', 
          schema = db_name, 
          con = engine,
          if_exists = 'append',
          index = False, 
          chunksize = 1000
  )

In [ ]:
drop_table = "DROP TABLE IF EXISTS music.tracks_played;"
engine.execute(drop_table)

create_table_sql = f'''
CREATE TABLE music.`tracks_played` (
  `user_id` smallint,
  `artist_id` smallint,
  `album_id` smallint,
  `track_id` smallint,
  `played_on` datetime,
  PRIMARY KEY(user_id, artist_id, album_id, track_id, played_on),
  FOREIGN KEY(user_id) REFERENCES users(user_id),
  FOREIGN KEY(artist_id, album_id, track_id) REFERENCES track(artist_id, album_id, track_id)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4
'''
engine.execute(create_table_sql)


In [ ]:
with engine.connect() as con:
  tracks = pd.read_sql("SELECT artist_id, album_id, track_id FROM music.track", con=con)

In [ ]:


def random_dates(start, end, n=10):

    start_u = start.value//10**9
    end_u = end.value//10**9

    return pd.to_datetime(np.random.randint(start_u, end_u, n), unit='s').sort_values()


In [ ]:

results = []
for d in range(1,30):
  start = pd.to_datetime(f'2022-12-{d}')
  end = pd.to_datetime(f'2022-12-{d+1}')
  for user_id in range(10):
    artist_listened = random.randint(1,6)
    tracks_listened = random.randint(1,10)
    listened = tracks.query(f"artist_id == {artist_listened}").sample(n=tracks_listened, replace=True)
    listened['user_id'] = user_id+1
    listened['played_on'] = random_dates(start, end, n = tracks_listened)
    results.append(listened)

    
df_listen = pd.concat(results)
df_listen.to_sql(
          name = 'tracks_played', 
          schema = db_name, 
          con = engine,
          if_exists = 'append',
          index = False, 
          chunksize = 1000
  )